<a href="https://colab.research.google.com/github/SujoySengupta/Agentic-AI-Sujoy/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# --- STEP 1: IMPORTS ---
import torch
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# --- STEP 2: CREATE DUMMY DATA (Bypassing the broken download) ---
# We manually create 3 "Research Papers" so the code has something to chew on.
raw_text_data = [
    {
        "text": "Deep Learning is a subset of machine learning using neural networks with three or more layers. These neural networks attempt to simulate the behavior of the human brain—albeit far from matching its ability—allowing it to 'learn' from large amounts of data. While a neural network with a single layer can still make approximate predictions, additional hidden layers can help to optimize and refine for accuracy.",
        "source": "Intro to DL.pdf"
    },
    {
        "text": "Retrieval-Augmented Generation (RAG) is a technique for enhancing the accuracy and reliability of generative AI models with facts fetched from external sources. By bridging the gap between private data and public LLMs, RAG solves the hallucination problem. It retrieves relevant context and passes it to the LLM along with the user query.",
        "source": "RAG Architecture.pdf"
    },
    {
        "text": "Agentic AI refers to AI systems that can pursue complex goals with limited direct supervision. Unlike passive AI which waits for prompts, agentic AI can break down tasks, use tools, and iterate on solutions. This autonomy allows for more complex workflows in software engineering and data analysis.",
        "source": "Agents_v1.pdf"
    }
]

print("Creating documents from dummy data...")
documents = [
    Document(page_content=item["text"], metadata={"source": item["source"]})
    for item in raw_text_data
]

# --- STEP 3: CHUNKING ---
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks = text_splitter.split_documents(documents)
print(f"Successfully created {len(chunks)} chunks.")

# --- STEP 4: EMBEDDINGS & VECTOR STORE ---
print("Creating embeddings...")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(chunks, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

# --- STEP 5: LLM SETUP ---
print("Initializing LLM...")
device = 0 if torch.cuda.is_available() else -1
hf_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_new_tokens=100,
    device=device
)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# --- STEP 6: RAG CHAIN ---
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# --- STEP 7: TEST QUERIES (For your Report) ---
print("\n--- TEST RESULTS ---")
queries = [
    "What is Retrieval-Augmented Generation?",
    "How is Agentic AI different from passive AI?",
    "What allows neural networks to learn?"
]

for q in queries:
    print(f"\nQ: {q}")
    response = rag_chain.invoke(q)
    print(f"A: {response}")

Creating documents from dummy data...
Successfully created 7 chunks.
Creating embeddings...
Initializing LLM...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
/tmp/ipython-input-108719738.py:56: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)



--- TEST RESULTS ---

Q: What is Retrieval-Augmented Generation?
A: a technique for enhancing the accuracy and reliability of generative AI models with facts fetched from external sources

Q: How is Agentic AI different from passive AI?
A: agentic AI can break down tasks, use tools, and iterate on Retrieval-Augmented Generation (RAG) is a technique for enhancing the accuracy and reliability of generative AI models with facts fetched from external sources

Q: What allows neural networks to learn?
A: large amounts of data


In [2]:
# Install compatible versions of torch and torchvision together
%pip install -U torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.0 MB/s eta 0:00:00
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.24.0+cpu
    Uninstalling torchvision-0.24.0+cpu:
      Successfully uninstalled torchvision-0.24.0+cpu
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.9.0+cpu
    Uninstalling torchaudio-2.9.0+cpu:
      Successfully uninstalled torchaudio-2.9.0+cpu
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.8.6 requires torch<2.10,>=1.10, but you have torch 2.10.0 which is incompatible.


In [2]:
%pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 76.4 MB/s eta 0:00:00
